In [1]:
import io
import re
import itertools

import geojson
import netCDF4
import pandas
import numpy as np


In [32]:
ds = netCDF4.Dataset('../data/transect.nc')

ids = ds.variables['id'][:]

id_ = 7003900

def transect2features(id_):
    transect_idx = np.searchsorted(ids, id_)

    variables = {
        'lat': {"var": 'lat', "slice": np.s_[transect_idx, :]},
        'lon': {"var": 'lon', "slice": np.s_[transect_idx, :]},
        'z': {"var": 'altitude', "slice": np.s_[:, transect_idx, :]},
        "t": {"var": 'time', "slice": np.s_[:]}

    }
    data = {}
    for var, props in variables.items():
        data[var] = ds.variables[props['var']][props['slice']]


    for t, row in zip(data['t'], data['z']):
        coords = pandas.DataFrame(data=dict(
            lon=data['lon'], 
            lat=data['lat'],
            z=row
        ))
        coordinates = coords.dropna().values.tolist()
        if not coordinates:
            continue
        geometry = geojson.LineString(coordinates=coordinates)
        date = netCDF4.num2date(t, ds.variables['time'].units)
        properties = {
            "date": date.isoformat(),
            "year": date.year
        }
        yield geojson.Feature(id="{}-{}".format(id_, date.year), geometry=geometry, properties=properties)


In [33]:
import itertools
import more_itertools

In [34]:
import tqdm

features = []
for id_ in tqdm.tqdm_notebook(ids):
    features.extend(transect2features(id_))



In [35]:
for i, chunk in enumerate(more_itertools.chunked(features, 10000)):
    collection = geojson.FeatureCollection(chunk)
    with open('jarkus{:03d}.json'.format(i), 'w') as f:
        geojson.dump(collection, f)


In [ ]:
%pdb off
